# Curso de Python do Zero ao DS
## Profº Meigarom
### Canal: Seja um Data Scientist


## Agenda

* 1.0 - O Problema do negócio
  * 1.1 - A Empresa: House Rocket
  * 1.2 - As perguntas do contratante (CEO)
* 2.0 - Planejamento da solução
  * 2.1 - Planejamento do produto final
  * 2.2 - Planejamento das ferramentas
  * 2.3 - Planejamento do processo
* 3.0 - Solução
* 4.0 - Entrega

# 1.0 - O Problema do negócio

## 1.1 - A Empresa: House Rocket
* O que faz? 
    * Plataforma de compras e vendas de imóveis.
* Qual é o problema? 
    * O CEO da empresa gostaria de maximizar os lucros encontrando bons negócios.
* Qual é a principal estratégia?
    * Fontes externas para encontrar bons negócios.

## 1.2 - As perguntas do contratante (CEO):

1. Qual a média do preço de compra dos imóveis por “Nível”? 
- Nível 0 -> Preço entre R$ 0 e R$ 321.950
- Nível 1 -> Preço entre R$ 321.950 e R$ 450.000
- Nível 2 -> Preço entre R$ 450.000 e R$ 645.000
- Nível 3 -> Acima de R$ 645.000

2. Qual a média do tamanho da sala de estar dos imóveis por “Size” ?
- Size 0 -> Tamanho entre 0 e 1427 sqft
- Size 1 -> Tamanho entre 1427 e 1910 sqft
- Size 2 -> Tamanho entre 1910 e 2550 sqft
- Size 3 -> Tamanho acima de 2550 sqft

3. Adicione as seguinte informações ao conjunto de dados original:
- Place ID: Identificação da localização 
- OSM Type: Open Street Map type
- Country: Nome do País
- Country Code: Código do País

4. Adicione os seguinte filtros no Mapa:
- Tamanho mínimo da área da sala de estar.
- Número mínimo de banheiros.
- Valor Máximo do Preço.
- Tamanho máximo da área do porão.
- Filtro das Condições do Imóvel.
- Filtro por Ano de Construção.

5. Adicione os seguinte filtros no Dashboard:
- Filtro por data disponível para compra.
- Filtro por ano de renovação.
- Filtro se possui vista para a água ou não.

# 2.0 - Planejamento da solução
## 2.1 - Planejamento do produto final
* Email+3 anexos:
  * Email: Pergunta | Resposta;
  * Anexo1: Arquivo .csv com as novas informações requisitadas;
  * Anexo2: Um mapa com os filtros requisitados;
  * Anexo3: Um dashboard com os filtros requisitados.

## 2.2 - Planejamento das ferramentas
* Python 3.9.2
* Jupyter Notebook
* VSCode

## 2.3 - Planejamento do processo

In [ ]:
# Carregar bibliotecas e dataset
import pandas as pd

In [ ]:
df = pd.read_csv('datasets/kc_house_data.csv')
df.dtypes


1. Qual a média do preço de compra dos imóveis por “Nível”? 
- Nível 0 -> Preço entre R$ 0 e R$ 321.950
- Nível 1 -> Preço entre R$ 321.950 e R$ 450.000
- Nível 2 -> Preço entre R$ 450.000 e R$ 645.000
- Nível 3 -> Acima de R$ 645.000

>>  Criar coluna 'nível' com a classificação mediante o intervalo de prços da instrução acima;


In [ ]:
df['nivel'] = 'NA'

In [ ]:
%%timeit
df['nivel'] = df['price'].apply( lambda x: 
                'Nivel 0'   if x < 321950.00 else 
                'Nivel 1'   if (x>=321950.00) & (x<450000.00) else
                'Nivel 2'   if (x>=450000.00) & (x<645000.00) else 
                'Nivel 3' )
df[['price','nivel']]

In [ ]:
%%timeit

df.loc[df['price']<321950.00,'nivel'] = 'Nivel 0'
df.loc[(df['price']>=321950.00) & (df['price']<450000.00),'nivel'] = 'Nivel 1'
df.loc[(df['price']>=450000.00) & (df['price']<645000.00),'nivel'] = 'Nivel 2'
df.loc[df['price']>=645000.00,'nivel'] = 'Nivel 3'

df[['price','nivel']]

>> Agrupar por 'nivel' e calcular a média dos preços.


In [ ]:
df[['price','nivel']].groupby('nivel').mean().reset_index()

2. Qual a média do tamanho da sala de estar dos imóveis por “Size” ?
- Size 0 -> Tamanho entre 0 e 1427 sqft
- Size 1 -> Tamanho entre 1427 e 1910 sqft
- Size 2 -> Tamanho entre 1910 e 2550 sqft
- Size 3 -> Tamanho acima de 2550 sqft

>> Criar coluna 'size' classificando os imóveis de acordo com a tabela de tamanho;
>>
>> Agrupar por 'size' e calcular a média do tamanho da sala.

In [ ]:
df['size'] = "NA"

In [ ]:
%%timeit
df.loc[df['sqft_living']<1427, 'size'] = 'Size 0'
df.loc[(df['sqft_living']>=1427) & (df['sqft_living']<1910),'size'] = 'Size 1'
df.loc[(df['sqft_living']>=1910) & (df['sqft_living']<2550),'size'] = 'Size 2'
df.loc[df['sqft_living']>2550, 'size'] = 'Size 3'
df[['sqft_living','size']]

3. Adicione as seguinte informações ao conjunto de dados original:
- Place ID: Identificação da localização 
- OSM Type: Open Street Map type
- Country: Nome do País
- Country Code: Código do País

In [ ]:
from geopy.geocoders import Nominatim

In [ ]:
# Inicializa Nominatim API
geolocator = Nominatim( user_agent='geoapiExercicios')

In [ ]:
df['place_id'] = 'NA'
df['osm_type'] = 'NA'
df['country'] = 'NA'
df['country_code'] = 'NA'

In [ ]:
df_len = int(len(df)/1000)
for i in range( df_len ):
    print('Loop {} / {}'.format(i, df_len))
    query = str(df['lat'][i])+','+str(df['long'][i])

    response = geolocator.reverse( query )
    response.raw['address']

    if 'country' in response.raw['address']:
        df.loc[i,'country'] = response.raw['address']['country']

    if 'country_code' in response.raw['address']:
        df.loc[i,'country_code'] = response.raw['address']['country_code']


In [ ]:
df.head()


4. Adicione os seguinte filtros no Mapa:
- Tamanho mínimo da área da sala de estar.
- Número mínimo de banheiros.
- Valor Máximo do Preço.
- Tamanho máximo da área do porão.
- Filtro das Condições do Imóvel.
- Filtro por Ano de Construção.

In [ ]:
df.dtypes

In [ ]:
import plotly.express as px
import ipywidgets as widgets
from ipywidgets import Layout
from ipywidgets import fixed

In [ ]:
style = {'description_width': 'initial'}
# Iteractive buttons
price_limit = widgets.IntSlider(
    value = int(df['price'].median()),
    min = int(df['price'].min()),
    max = int(df['price'].max()),
    step = int(df['price'].median()/10),
    description='Maximun Price',
    disable=False,
    style=style
)

In [ ]:
df['bedrooms'].unique().tolist()

In [ ]:
bedrooms_bar = widgets.Dropdown(
    options = df['bedrooms'].unique().tolist(),
    value = 3,
    description = 'Bedrooms',
    disable=False
)

In [ ]:
def update_map( df, bedrooms, limit ):
    filtro = ((df['price'] <= limit) & (df['bedrooms'] == bedrooms))
    campos = ['id', 'lat', 'long', 'price', 'bedrooms','nivel']
    df = df[filtro][campos]
    
    fig = px.scatter_mapbox( df, 
                             lat="lat", 
                             lon="long", 
                             size="price",
                             color="nivel",
                             color_continuous_scale=px.colors.cyclical.IceFire, 
                             size_max=15, 
                             zoom=10)
    fig.update_layout(mapbox_style="open-street-map")
    fig.update_layout(height=600, margin={"r":0,"t":0,"l":0,"b":0})
    fig.show()



In [ ]:
widgets.interactive( update_map, df=fixed( df ), bedrooms=bedrooms_bar, limit=price_limit )

5. Adicione os seguinte filtros no Dashboard:
- Filtro por data disponível para compra.
- Filtro por ano de renovação.
- Filtro se possui vista para a água ou não.

In [ ]:
from matplotlib import gridspec
from matplotlib import pyplot as plt 

In [ ]:
df.dtypes

In [ ]:
# Change data format
df['year']      = pd.to_datetime( df['date'] ).dt.strftime( '%Y' )
df['date']      = pd.to_datetime( df['date'] ).dt.strftime( '%Y-%m-%d' )
df['year_week'] = pd.to_datetime( df['date'] ).dt.strftime( '%Y-%U' )
df['yr_built']  = pd.to_datetime( df['yr_built'] ).dt.strftime( '%Y-%m-%d' )
df['yr_renovated']  = pd.to_datetime( df['yr_renovated'] ).dt.strftime( '%Y-%m-%d' )
df[['date','year','year_week','yr_renovated','yr_built']].head()